In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# get the table info
r  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", "lxml")

data = r.text

soup = BeautifulSoup(data)

table = soup.find_all('table')[0] 

# convert list to dataframe
dfs = pd.read_html(str(table))

df = pd.DataFrame(dfs[0])


In [13]:
# set the first row as columns
new_header = df.iloc[0]
df=df[1:]
df.columns = new_header
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [14]:
#drop the invalid rows
indexRows = df[df['Borough'] == 'Not assigned'].index
df.drop(indexRows, inplace=True)

In [15]:
# group the rows with same postcode, and list all neighbourhood
grouped_df = df.groupby(['Postcode','Borough'])
grouped_lists = grouped_df['Neighbourhood'].agg(lambda x: ",".join(x))
grouped_lists = grouped_lists.reset_index()
grouped_lists.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
grouped_lists['Neighbourhood'].replace({"Not assigned": grouped_lists['Borough']}, inplace=True)

In [17]:
# get the number of rows
grouped_lists.shape

(103, 3)